# This notebook walks through a simple usage of the grismconf software with NIRISS configurations
The reference software is stored here: https://github.com/npirzkal/grismconf

* Configuration files in the style of aXe were created by the team to relate the dispersion polynomials.
* This software, along withe configuration and sensitivity files, was used to validate the GWCS models created for use with the JWST pipeline WFSS mode
* The corresponding reference files that were created for NIRCAM were made using the code in this repository: https://github.com/spacetelescope/jwreftools/blob/master/jwreftools/niriss/niriss_grism_reffiles.py

*In order to continue with this notebook, you'll need a copy of the teams aXe style conf files and the sensitivity files*

`GR150C.F090W.t.conf` is an example of one such file. They were made to work in conjunction with the sensitivity files, such as `NIRISS.GR150C.F090W.p1.etc.sens.fits`. You'll need to grab a copy of both sets of files to continue with this notebook.

## NIRISS

In [ ]:
import grismconf
from grismconf import poly
import os

In [ ]:
os.chdir('grismconf_files')

### This is what an example aXe style conf file contains:

In [ ]:
!cat GR150C.F090W.t.conf

**You can see above that the file contains information for orders "A" through "E", for the column dispersed grism**

### The config files can be read by grismconf as follows, the data will be stored to a python object for later use

In [ ]:
N=grismconf.Config('GR150C.F090W.t.conf')

In [ ]:
print("Wavelength range: {}".format(N._DISPL_data['A']))
print("X coefficients: {}".format(N._DISPX_data['A']))
print("Y coefficients: {}".format(N._DISPY_data['A']))

#### t is a free variable 0<t<1


In [ ]:
order = "A"
x = 1000
y = 1000
l = 15000

In [ ]:
t=grismconf.poly.INVPOLY10(N._DISPL_data[order], x, y, l)  # call the wavelength dispersion model at location 1000,1000 and lam=25000
print("t:",t)

In [ ]:
grismconf.poly.DPOLY10(N._DISPX_data[order], x, y ,t)  # call one of the x-models with location 1000,1000 and the t you calculated

In [ ]:
N.DISPX(order, x, y, t)  # x-offset

In [ ]:
grismconf.poly.DPOLY10(N._DISPY_data[order], x, y, t)  # dispersion is 2D dependent

In [ ]:
grismconf.poly.DPOLY10(N._DISPX_data[order], x, y, t)  # dispersion is 2D dependent

### We have a pixel at x,y = (1000,1000) in the grism data which is known to be at wavelength l=1.0micron, what is the origin of this light in the direct image?
**NIRISS dispersion currently has 2D field dependence**
​


In [ ]:
x = 1000
y = 1000
l = 10000
order = "A"

# Initial guess for t
t = N.INVDISPL(order,x,y,l)

# We compute the dx and dy values for that t value
dx = N.DISPX(order,x,y,t)
dy = N.DISPY(order,x,y,t)

print("dx:",dx)
print("dy:",dy)
print("t:", t)
print("Light at ({} Angstroms) originating from object at: ({}, {}) in the direct image".format(l, x+dx, y+dy))

### Let's reverse the calculation, using the location in the direct image and wavelength, where is the pixel in the grism image?

In [ ]:
x = 1010.0284864061894
y = 997.6842400874059
l = 10000 # angstroms
order = "A"

t = N.INVDISPL(order, x, y, l)
dx = N.DISPX(order, x, y, t)
dy = N.DISPY(order, x, y, t)

print("dx:",dx)
print("dy:",dy)
print("t:", t)
print("Light at ({} Angstroms) comes from pixel: ({}, {}) in the dispersed image".format(l, x-dx, y-dy))


## The NIRISS grism dispersions also have a dependence on the filter wheel rotation, so an additional rotation needs to be added to the calculation to get the correction translation

The reference filter wheel position is taken from the `FWCPOS_REF` value in the `specwcs` reference file, the filter wheel position for the observation being processed is taken from the `FWCPOS` value in the meta information. These are combined such that the angle of rotation, theta, is `FWCPOS_REF` - `FWCPOS`.

### We have a pixel at x,y = (1000,1000) in the grism data which is known to be at wavelength l=1.0micron, what is the origin of this light in the direct image?
NIRISS dispersion currently has 2D field dependence and requires a known rotatio, below we will add the rotation part with an example theta


In [ ]:
import numpy as np
x = 1000
y = 1000
l = 10000
dx = 0
dy = 0
order = "A"
theta = 324.5 * np.pi / 180.

# Initial guess for t
t = N.INVDISPXY(order, x, y, dx=0, dy=0, theta=theta)
print("t: ", t)

# We compute the dx and dy values for that t value
dx, dy = N.DISPXY(order, x, y, t)

print("dx:",dx)
print("dy:",dy)
print("Light at ({} Angstroms) originating from object at: ({}, {}) in the direct image".format(l, x+dx, y+dy))

### Using the location in the direct image and wavelength, where is the pixel in the grism image?

In [ ]:
x = 998.3664307111518
y = 997.7098203215904
l = 10000 # angstroms
dx = 0
dy = 0
theta = 324.5 * np.pi / 180.

order = "A"

t = N.INVDISPXY(order, x, y, dx=dx, dy=dy, theta=theta)
dx, dy = N.DISPXY(order, x, y, t)

print("dx:",dx)
print("dy:",dy)
print("t:", t)
print("Light at ({} Angstroms) comes from pixel: ({}, {}) in the dispersed image".format(l, x-dx, y-dy))